# Basic Flowsheet Build Tutorial

This tutorial will go through the basic workflow to create a WaterTAP model with a single unit process (pump).

1. Required imports
2. Build model
   1. Create model object
   2. Add flowsheet
   3. Add property model
   4. Add pump model
3. Set operating conditions
4. Scaling model
5. Initialize model
6. Solve model
7. Analyze/Visualize results
   
<p align="center">
<img src="img/wf_7steps.svg" alt="Basic Worflow" width="500">
</p>

## Step 1: Import necessary packages

### Any WaterTAP model will begin with importing the relevant components from Pyomo, IDAES, and WaterTAP.

From Pyomo:
- Concrete model
- Helper function(s) to check solver status
- Helper function to get value of variables

From IDAES:
- Flowsheet block
- Helper function to check degrees of freedom
- Helper function to scale model

From WaterTAP:
- Property models
- Unit models
- Solver


<p align="center">
<img src="img/wf_7steps_1.svg" alt="Basic Worflow" width="500">
</p>

<div class="alert alert-block alert-info">
<b>Note:</b> Both <code>ConcreteModel</code> and <code>FlowsheetBlock</code> are required imports for any WaterTAP model.
</div>

In [ ]:
# Pyomo imports
from pyomo.environ import (
    ConcreteModel,
    check_optimal_termination,
    assert_optimal_termination,
    value,
)

# IDAES imports
from idaes.core import FlowsheetBlock
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.scaling import calculate_scaling_factors

# WaterTAP imports
from watertap.core.solvers import get_solver
from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock
from watertap.unit_models.pressure_changer import Pump

# Step 2: Build Model

<p align="center">
<img src="img/wf_7steps_2.svg" alt="Basic Worflow" width="500">
</p>

## Step 2.1: Create base model

<p align="center">
<img src="img/wf_7steps_2-1.svg" alt="Basic Worflow" width="500">
</p>

Pyomo's `ConcreteModel` is the basis for any WaterTAP model.

No arguments are required. Convention is to call this `m`.

In [ ]:
m = ConcreteModel()

## Step 2.2: Add flowsheet block

<p align="center">
<img src="img/wf_7steps_2-2.svg" alt="Basic Worflow" width="500">
</p>

The [`FlowsheetBlock`](https://idaes-pse.readthedocs.io/en/stable/reference_guides/core/flowsheet_block.html#) from IDAES forms the base for all WaterTAP system models.

All model components (unit models, property models, variables, constraints, etc.) are built upon the flowsheet block.

The flowsheet block will create the time domain for the flowsheet. We set `dynamic=False` when creating the `FlowsheetBlock` to represent a steady-state flowsheet.

(Note: `dynamic=False` is the default argument, but excluding it will raise a warning.)

In [ ]:
m.fs = FlowsheetBlock(dynamic=False)

## Step 2.3: Add property model

With the flowsheet created, we can add any sub-models required for the WaterTAP system model.

<p align="center">
<img src="img/wf_7steps_2-3.svg" alt="Basic Worflow" width="500">
</p>

The property model used will be dictated by the models on the flowsheet. In this demonstration, we are using the WaterTAP [seawater property model](https://watertap.readthedocs.io/en/stable/technical_reference/property_models/seawater.html).

Creating the `SeawaterParameterBlock` does not require any arguments. However, other property models can have many required and optional arguments to specify the model.

In [ ]:
m.fs.properties = SeawaterParameterBlock()

## Step 2.4: Add unit model

After adding the property model, we can add the unit models to the flowsheet. In this demonstration, we are adding a single [pump model](https://watertap.readthedocs.io/en/stable/technical_reference/unit_models/pump.html).

<p align="center">
<img src="img/wf_7steps_2-4.svg" alt="Basic Worflow" width="500">
</p>

Adding any unit model requires specifying the property package via the `property_package` argument. Here, we are specifying that the property model to be used with our pump model is `m.fs.properties` that we just added to the flowsheet.

In [ ]:
m.fs.pump = Pump(
    property_package=m.fs.properties,
)

# Step 3: Set operating conditions

Now that we have added all the necessary models to the flowsheet, we can specify the operating conditions.

<p align="center">
<img src="img/wf_7steps_3.svg" alt="Basic Worflow" width="500">
</p>

First, we check the degrees of freedom that need to be specified prior to solving the model. Degrees of freedom for individual unit models can be found on the WaterTAP documentation.


In [ ]:
print("Degrees of freedom =", degrees_of_freedom(m))

### The degrees of freedom for the pump model include:
- the change in pressure from inlet to outlet
- pump efficiency
- inlet stream state variables

The inlet stream conditions to any unit model are defined by the property model state variables. For the pump model, the inlet and outlet properties are managed by a [control volume](https://idaes-pse.readthedocs.io/en/stable/reference_guides/core/control_volume_0d.html#module-idaes.core.base.control_volume0d).

We can use the `.display()` method on the pump unit model control volume. Using this method on any block will automatically go through all the sub-blocks and show all the variables, constraints, and objectives currently on each block.

In [ ]:
m.fs.pump.control_volume.display()

### This reveals that on `m.fs.pump.control_volume` are two sub-blocks called `properties_in` and `properties_out`. 

Each of these are indexed to `0`, which is the default time-index for steady-state models.

To define the inlet stream to our pump, we only need to specify the state variables for `properties_in`. The state variables are defined by the property model used for the unit model (here that is the `SeawaterParameterBlock`) but are nearly always:
- `pressure`: pressure of inlet stream (Pa)
- `temperature`: temperature of (K)
- `flow_mass_phase_comp["Liq", "H2O"]`: mass flow rate of water (kg/s)
- `flow_mass_phase_comp["Liq", "TDS"]`: mass flow rate of TDS (kg/s)

<div class="alert alert-block alert-info">
<b>Note:</b> The naming of property variables in WaterTAP property models follow the <a href="https://idaes-pse.readthedocs.io/en/latest/explanations/conventions.html#standard-naming-format">naming conventions used in IDAES</a>.
</div>

In [ ]:
m.fs.pump.control_volume.properties_in[0].pressure.fix(101325)  # Pa
m.fs.pump.control_volume.properties_in[0].temperature.fix(298.15)  # K
m.fs.pump.control_volume.properties_in[0].flow_mass_phase_comp["Liq", "H2O"].fix(
    1
)  # kg/s
m.fs.pump.control_volume.properties_in[0].flow_mass_phase_comp["Liq", "TDS"].fix(
    0.1
)  # kg/s

print("Degrees of freedom =", degrees_of_freedom(m))

### With the inlet stream specified, the remaining two degrees of freedom are variables that are built directly on the pump block itself (rather than on the control volume sub-block):

In [ ]:
m.fs.pump.deltaP.fix(500000)  # Pa
m.fs.pump.efficiency_pump.fix(0.8)

print("Degrees of freedom =", degrees_of_freedom(m))

# Step 4: Scaling the model

Prior to model initialization and solving, the variables and constraints on the model must be properly scaled.

<p align="center">
<img src='img/wf_7steps_4.svg' alt="Basic Worflow" width="500">
</p>

Model scaling is a topic that will be covered in more detail in a future class, but here we present the helper function `calculate_scaling_factors` that will use default scaling factors to scale the variables on the model.

<div class="alert alert-block alert-info">
<b>Note:</b> Since we did not set default scaling factors for some of our variables, this will result in printed warnings.
</div>

In [ ]:
calculate_scaling_factors(m)

# Step 5: Initializing the model

Before solving the model, we run an initialization routine that will set model variables to initial values based on the operating conditions we have specified in the previous step.

<p align="center">
<img src='img/wf_7steps_5.svg' alt="Basic Worflow" width="500">
</p>


Every model has a different initialization routine that was implemented by the model developer, but they can all be called with the `initialize()` method.

In [ ]:
m.fs.pump.initialize()

# Step 6: Solving the model

After initialization, we are ready to solve the model.

<p align="center">
<img src='img/wf_7steps_6.svg' alt="Basic Worflow" width="500">
</p>


Solving is done by passing the model to a *solver*. WaterTAP uses a custom implementation of the IPOPT solver. An instance of the WaterTAP solver is returned from the `get_solver()` function that was imported above.

In [ ]:
solver = get_solver()
results = solver.solve(m)

### Checking the termination status of the solve

We want to know if the solve resulted in an optimal termination. This can be done in a few ways:
- using the `assert_optimal_termination` or `check_optimal_termination` function
- printing the solver status from the `results`

In [ ]:
# This will raise an error if the model did not solve optimally
assert_optimal_termination(results)

# Alternatively, we can check the termination status without raising an error
termination_status = check_optimal_termination(results)
print("Optimal termination:", termination_status)

# Printing termination condition from results
print("Solver termination condition:", results.solver.termination_condition)

# Step 7: Accessing results

<p align="center">
<img src='img/wf_7steps_7.svg' alt="Basic Worflow" width="500">
</p>

The values of all variables on the model can be accessed directly by using the `value()` function.

In [ ]:
pump_work = value(m.fs.pump.work_mechanical[0])

print(f"Pump work: {pump_work:.2f} W")

Alternatively, most unit models in WaterTAP and IDAES have a `report()` method.

In [ ]:
m.fs.pump.report()